In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib ipympl
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['font.size'] = 8
mpl.rc('image', cmap='gray')
gs = gridspec.GridSpec(2, 2)

import matplotlib.animation
writervideo = matplotlib.animation.FFMpegWriter(fps=30)

import numpy as np
import pandas as pd
import csv, json

import pims
import trackpy as tp
tp.quiet()
from PIL import Image, ImageDraw
import cv2

from scipy.optimize import dual_annealing, minimize
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm
import joblib

from scipy.spatial import distance_matrix
from scipy.ndimage import uniform_filter1d

import random

run_analysis_verb = False
show_verb = True
save_verb = True

# TRACKPY TRACKING 
Since in movie.mp4 at frame merge_frame = 32269 two droplets merge together, the tracking procedure is divided into two parts: \
    - Pre merge [0:merge_frame] --> 50 features to locate \
    - Post merge [merge_frame:] --> 49 features to locate

In [ ]:
# PRE PROCESSING
@pims.pipeline
def trackpy_preprocessing(image, x1, y1, x2, y2):    
    npImage = np.array(image)
    # Create same size alpha layer with circle
    #alpha = Image.new('L', (920, 960), 0)
    alpha = Image.new('L', (920, 960), 0)

    draw = ImageDraw.Draw(alpha)
    draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)

    # Convert alpha Image to numpy arrayf
    npAlpha = np.array(alpha)
    npImage = npImage[:, :, 1] * npAlpha
    
    ind = np.where(npImage == 0)
    # npImage[200, 200] color of the border to swap with the black
    npImage[ind] = npImage[200, 200]
    return npImage

In [ ]:
# import video with pre processing applied
data = trackpy_preprocessing(pims.open('./data/movie.mp4'), 55, 55, 880, 880)
data_ref = pims.open('./data/movie.mp4')

In [ ]:
frame = 0
fig, (ax, ax1) = plt.subplots(1, 2)
ax.imshow(data_ref[frame])
ax.set(title = f'Original - frame {frame}', xlabel = 'X [px]', ylabel = 'Y [px]')
ax1.imshow(data[frame])
ax1.set(title = f'Processed - frame {frame}', xlabel = 'X [px]', ylabel = 'Y [px]')
plt.tight_layout()
if save_verb: plt.savefig('./results/pre_processing_example.png', bbox_inches='tight')
if show_verb: 
    plt.show()
else:
    plt.close()

## PRE MERGE 

In [ ]:
run_analysis_verb = False
if run_analysis_verb:
    ###############################################################################################
    #                                           PARAMETERS                                        #
    ###############################################################################################
    dropSize = 31  
    minMass = 2000
    sep = 16
    nDrops = 50
    trackpy_params = {"dropSize": dropSize, "minMass": minMass, "sep": sep, "nDrops": nDrops}
    # save parameters used for the trial tracking:
    with open('./results/tracking_data/trackpy_trial_params.txt', 'w') as fp:
        json.dump(trackpy_params, fp)

    ###############################################################################################
    #                                     FEATURES LOCATION                                       #
    ###############################################################################################
    nFrames = 100 # up to merge_frame
    startFrame = 0
    endFrame = startFrame + nFrames
    f = tp.batch(data[startFrame:endFrame], dropSize, minmass = minMass, separation = sep, topn = nDrops, engine = 'numba')    
    display(f)

    ###############################################################################################
    #                                      FEATURES LINKING                                       #
    ###############################################################################################
    t = tp.link_df(f, 150, memory = 2, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
    display(t)

    # CREATE COLOR COLUMN AND SAVE DF
    n = max(t.particle)
    print(n)
    random.seed(5)
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
    for i in range(max(t.particle)+1-n):
        colors.append("#00FFFF")
    c = []
    for p in t.particle:
        c.append(colors[p])
    t["color"] = c
    trajectory = t.copy()
    trajectory.to_parquet('./results/tracking_data/trackpy_trial.parquet')
    display(trajectory)
else:
    try:
        trajectory = pd.read_parquet('./results/tracking_data/trackpy_pre_merge_sorted_and_colored.parquet')
        display(trajectory)
    except:
        print("ERROR: no trajectory data found")

In [ ]:
"""
# need to work on this

# ANOMALIES DETECTION
num = np.zeros(1000)
for i in range(1000):
    num[i] = len(f.loc[f['frame'] == i + startFrame])

idx = np.where(num != nDrops)[0]
print(idx)
if len(idx) != 0:
    delta = np.zeros(len(idx)-1)
    for i in range(len(idx)-1):
        delta[i] = idx[i+1]-idx[i]
    print(delta)

t = tp.link_df(f, 50, memory = 2, link_strategy = 'hybrid', neighbor_strategy = 'KDTree', adaptive_stop = 1)
t = tp.filter_stubs(t, 50)

# ANOMALIES DETECTION
n = max(t.particle)
print(n)
random.seed(5)
colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(n)]
for i in range(max(t.particle)+1-n):
    colors.append("#00FFFF")
c = []
for p in t.particle:
    c.append(colors[p])
t["color"] = c

display(t)
df = t.loc[(t.frame == startFrame+100) , ["x", "y", "color"]]
fig, ax = plt.subplots(1, 1)
ax.scatter(df.x, df.y, facecolors = 'none', edgecolors = df.color, s = 50)
ax.imshow(data_preload[100])
plt.show()
df = t.loc[t['frame'] == f1, ["x", "y", "color", "particle"]]

fig, (ax, ax1) = plt.subplots(1, 2)
ax.scatter(df.x, df.y, s=50, ec = "w", facecolor = df.color)
ax.imshow(data[f1])
df1 = t.loc[t['frame'] == f2, ["x", "y", "color"]]
ax1.scatter(df1.x, df1.y, s=50, ec = "w", facecolor = df1.color)
ax1.imshow(data[f2])
plt.show()
lost_particles = []
for i in range(nFrames-1):
    a = t.loc[t.frame == i].sort_values('particle').particle.values
    b = t.loc[t.frame == i+1].sort_values('particle').particle.values
    ind = np.where(np.in1d(a, b)==False)[0]
    if ind.size > 0:
        lost_particles.append([i, a[ind]])
print("Total problems:", len(lost_particles))
print(ind)

if len(lost_particles) > 0: 
    print(f"First failure occurred at frame {lost_particles[0][0]}")
    f1 = lost_particles[0][0]
    f2 = lost_particles[0][0]+1
    print(t.loc[t.frame == f1].sort_values('particle').particle.values)
    print(t.loc[t.frame == f2].sort_values('particle').particle.values)

if len(lost_particles)>0: 
    fig, (ax, ax1) = plt.subplots(1, 2)
    df = t.loc[t['frame'] == f1, ["x", "y", "color", "particle"]]
    ax.scatter(df.x, df.y, s=50, ec = "w", facecolor = df.color)
    #ax.scatter(df.loc[df.particle==43].x, df.loc[df.particle==43].y, s=100)
    #ax.scatter(df.loc[df.particle==82].x, df.loc[df.particle==82].y, s=100)
    ax.imshow(data[f1])

    df1 = t.loc[t['frame'] == f2, ["x", "y", "color"]]
    ax1.scatter(df1.x, df1.y, s=50, ec = "w", facecolor = df1.color)
    ax1.imshow(data[f2])

    plt.show()
"""

In [ ]:
# animated plot of the tracking results
fig = plt.figure(figsize = (5, 5))
anim_running = True

def onClick(event):
    global anim_running
    if anim_running:
        ani.event_source.stop()
        anim_running = False
    else:
        ani.event_source.start()
        anim_running = True

def update_graph(frame):
    df = trajectory.loc[(trajectory.frame == frame) , ["x","y","color","size"]]
    for i in range(50):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = 2*df["size"].values[i]
    graph2.set_data(data[frame])
    title.set_text('Tracking raw - frame = {}'.format(frame))
    return graph

ax = fig.add_subplot(111)
title = ax.set_title('Tracking raw - frame = 0')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = trajectory.loc[(trajectory.frame == 0), ["x","y","color","size"]]

graph = []
for i in range(50):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), 2*df["size"].values[i], color = df.color.values[i],\
                                           fill = False, linewidth=1)))
graph2 = ax.imshow(data[0])

fig.canvas.mpl_connect('button_press_event', onClick)
ani = matplotlib.animation.FuncAnimation(fig, update_graph, range(0, max(trajectory.frame), 1), interval = 5, blit=False)
if 0: 
    writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
    ani.save('./results/tracking_raw_trial.mp4', writer=writer, dpi = 300)
plt.show()

## POST MERGE  - to do

In [ ]:
# parameters of the tracking
dropSize = 31  
minMass = 2000
sep = 16
nDrops = 49

# FEATURES LOCATION
nFrames = 100 # total number of frames of video --> len(data)
startFrame = merge_frame + 100 
endFrame = startFrame + nFrames

preload = True
if preload:
    data_preload = list(data[startFrame:endFrame])

In [ ]:
f = tp.batch(data_preload, dropSize, minmass = minMass, separation = sep, topn = nDrops, engine = 'numba')    
f.frame = f.frame + startFrame

# HOUGH TRACKING

In [4]:
@pims.pipeline
def hough_preprocessing(image, x1, y1, x2, y2):    
    #image = cv2.GaussianBlur(image, ksize = [7,7], sigmaX = 1.5, sigmaY = 1.5)
    npImage = np.array(image)
    # Create same size alpha layer with circle
    alpha = Image.new('L', (920, 960), 0)

    draw = ImageDraw.Draw(alpha)
    draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)

    # Convert alpha Image to numpy array
    npAlpha = np.array(alpha)
    npImage = npImage[:, :, 1] * npAlpha
    
    ind = np.where(npImage == 0)
    # npImage[200, 200] color of the border to swap with the black
    npImage[ind] = npImage[200, 200]
    npImage = cv2.medianBlur(npImage, 5)
    return npImage

@joblib.delayed
def loc_frame(correct_n, frame, img, parameters):
	temp = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **parameters)
	if (temp is not None) and (temp.shape[1] == correct_n):
		return np.hstack((temp[0], (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*correct_n))
	elif (temp is not None) and (temp.shape[1] != correct_n):
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*temp.shape[1]))
	else:
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.zeros((correct_n, 1), dtype=int)))


In [ ]:
def hough_feature_location(data_preload, frames, correct_n, params):
    parallel = joblib.Parallel(n_jobs = -2)
    temp = parallel(
        loc_frame(correct_n, frame, data_preload[frame-frames[0]], params)
        for frame in frames #tqdm(frames)
    )
    temp = pd.DataFrame(np.array(temp).reshape(len(frames)*correct_n, 5), columns = ["x", "y", "d", "frame", "nDroplets"])
    err_frames = frames[0] + np.where(temp.groupby("frame").mean().nDroplets!=correct_n)[0]
    loss = err_frames.shape[0]/frames.shape[0]
    return temp, err_frames, loss

In [ ]:
def optimize_params(x, *args):
    data_preload, frames, correct_n, writer = args
    params = {"dp":x[0], "minDist":x[1], "param1":x[2], "param2":x[3], "minRadius":int(x[4]), "maxRadius":int(x[5])}
    _, _, loss = hough_feature_location(data_preload, frames, correct_n, params)
    # write to csv loss and parameters
    writer.writerow([loss] + list(params.values()))
    return loss

def plot_optimization_results(opt_result_df, slot2):
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.arange(0, len(opt_result_df.loss), 1), opt_result_df.loss, 'b-')
    ax.set_ylabel("loss", color = 'b') 
    ax1 = ax.twinx() 
    ax1.plot(np.arange(0, len(opt_result_df[slot2]), 1), opt_result_df[slot2], 'r.')
    ax1.set_ylabel(slot2, color='r')
    ax.grid()
    plt.show()
    return fig

## PRE MERGE

In [ ]:
# SETUP
preload_load_data = False # takes 20 min
merge_frame = 32269
data = hough_preprocessing(pims.open('./data/movie.mp4'), 40, 55, 895, 910)
if preload_load_data: data_preload = list(data[:merge_frame])

startFrame = 0
endFrame = merge_frame
frames = np.arange(startFrame, endFrame, 100)
correct_n = 50
default_parameters = {"dp": 1.5, "minDist": 15, "param1": 100, "param2": 0.8, "minRadius": 15, "maxRadius": 25}

In [ ]:
optimization_verb = False
run_optimization_verb = False
if optimization_verb:
    if run_optimization_verb:
        with open("./results/tracking_data/hough/pre_merge_optimization.csv", 'w') as f1:
            writer = csv.writer(f1, delimiter='\t',lineterminator='\n',)
            writer.writerow(["loss", "dp", "minDist", "param1", "param2", "minRadius", "maxRadius"])
            opt_result = dual_annealing(optimize_params, x0 = [1.5, 15, 100, 0.9, 15, 25], \
                                        args = (data_preload, frames, correct_n, writer),\
                                        bounds = [(1, 3), (5, 20), (50, 200), (0.3, 1), (5, 20), (10, 30)],\
                                        maxiter = 2000)
    else:
        try:
            opt_result_df = pd.read_csv("./results/tracking_data/hough/pre_merge_optimization.csv", sep="\t").sort_values("loss", ascending=False)
        except:
            raise Exception("No optimization results found")

        optimized_parameters = opt_result_df.iloc[-1]
        optimized_parameters = {"dp": optimized_parameters.dp, "minDist": optimized_parameters.minDist,\
                                "param1": optimized_parameters.param1, "param2": optimized_parameters.param2,\
                                "minRadius": int(optimized_parameters.minRadius), "maxRadius": int(optimized_parameters.maxRadius)}
        parameters = optimized_parameters
        print(parameters)
else:
    parameters = default_parameters

In [ ]:
if run_analysis_verb:
    # save to txt parameters:
    with open('./results/tracking_data/hough/hough_pre_merge.txt', 'w') as f:
        f.write(json.dumps(parameters))
    pre_merge_df, err_frames, error = hough_feature_location(data_preload, frames, correct_n, parameters)
    pre_merge_df.to_parquet("./results/tracking_data/hough_pre_merge.parquet")
else:
    try:
        parameters = json.load(open('./results/tracking_data/hough/hough_pre_merge.txt'))
        post_merge_df = pd.read_parquet("./results/tracking_data/hough/hough_pre_merge.parquet")
        print(parameters)
        display(post_merge_df)
    except:
        raise Exception("No pre merge data found, run analysis first")

## POST MERGE 

In [ ]:
# SETUP
preload_load_data = False # takes 20 min
merge_frame = 32269
data = hough_preprocessing(pims.open('./data/movie.mp4'), 40, 55, 895, 910)
if preload_load_data: data_preload = list(data[merge_frame:])

startFrame = merge_frame
endFrame = len(data) 
frames = np.arange(startFrame, endFrame, 1)
opt_test_frames = np.arange(startFrame, endFrame, 500)
correct_n = 49
default_parameters = {"dp": 1.5, "minDist": 10, "param1": 100, "param2": 0.8, "minRadius": 10, "maxRadius": 30}

In [ ]:
optimization_verb = True
run_optimization_verb = False
if optimization_verb:
    if run_optimization_verb:
        with open("./results/tracking_data/hough/post_merge_optimization.csv", 'w') as f1:
            writer = csv.writer(f1, delimiter='\t',lineterminator='\n',)
            writer.writerow(["loss", "dp", "minDist", "param1", "param2", "minRadius", "maxRadius"])
            opt_result = dual_annealing(optimize_params, x0 = [1.5, 15, 100, 0.9, 15, 25], \
                                        args = (data_preload, opt_test_frames, correct_n, writer),\
                                        bounds = [(1, 3), (5, 20), (50, 200), (0.3, 1), (5, 20), (10, 30)],\
                                        maxiter = 2000)
    else:
        opt_result_df = pd.read_csv("./results/tracking_data/hough/post_merge_optimization.csv", sep="\t").sort_values("loss", ascending=False)
        # plot example of optimization result, in this case minRadius, can be any parameter
        plot_optimization_results(opt_result_df, "minRadius")
        
        optimized_parameters = opt_result_df.iloc[-1]
        optimized_parameters = {"dp": optimized_parameters.dp, "minDist": optimized_parameters.minDist,\
                                "param1": optimized_parameters.param1, "param2": optimized_parameters.param2,\
                                "minRadius": int(optimized_parameters.minRadius), "maxRadius": int(optimized_parameters.maxRadius)}
        parameters = optimized_parameters
        print(parameters)
else:
    parameters = default_parameters

In [ ]:
if run_analysis_verb:
    # save to txt parameters:
    with open('./results/tracking_data/hough/hough_post_merge.txt', 'w') as f:
        f.write(json.dumps(parameters))
    post_merge_df, err_frames, error = hough_feature_location(data_preload, frames, correct_n, parameters)
    post_merge_df.to_parquet("./results/tracking_data/hough/hough_post_merge.parquet")
else:
    try:
        parameters = json.load(open('./results/tracking_data/hough/hough_post_merge.txt'))
        post_merge_df = pd.read_parquet("./results/tracking_data/hough/hough_post_merge.parquet")
        print(parameters)
        display(post_merge_df)
    except:
        raise Exception("No post merge data found, run analysis first")

In [ ]:
# example of a frame with errors
problem_frame = frames[np.where(post_merge_df.groupby("frame").mean().nDroplets == 48)[0][-1]] #- merge_frame
example = cv2.HoughCircles(data_preload[problem_frame], cv2.HOUGH_GRADIENT_ALT, **parameters)

fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(gs[0, 1]) 
ax1.plot(frames, test_result.groupby("frame").mean().nDroplets, 'b-')
ax1.set(title = "Detected droplets", ylabel = "n")
ax1.grid()

ax2 = fig.add_subplot(gs[1, 1]) 
ax2.plot(frames, test_result.d.values.reshape(len(frames), correct_n), 'b.', markersize=1)
ax2.set(xlabel = "Frame", ylabel = "d [px]", title = "Droplets diameter")
ax2.grid()

ax3 = fig.add_subplot(gs[:, 0]) 
ax3.imshow(data_preload[problem_frame], cmap = "gray")
ax3.set(title=f"Detected droplets: {example.shape[1]} - Frame: {problem_frame + merge_frame}", xlabel = "X [px]", ylabel = "Y [px]")
for i in range(example.shape[1]):
    ax3.add_patch(plt.Circle((example[0][i][0], example[0][i][1]), example[0][i][2], color = "r", fill = False))
plt.tight_layout()
plt.show()

# DIMENSION OF DROPLETS ANALYSIS

## TRACKPY

In [ ]:
rawTrajs = pd.read_parquet("./results/tracking_data/trackpy_full.parquet")
mean_dim = rawTrajs.groupby("frame").mean()
merge_frame = 32269

fig, ax = plt.subplots(1, 1)
ax.plot(mean_dim.index/10, 2*mean_dim["size"])
ax.vlines(merge_frame/10, 2*mean_dim["size"].values.min(), 2*mean_dim["size"].max(), color="r")
ax.set(xlabel = "Time [s]", ylabel = "d [px]", title = "Mean particle diameter")
ax.grid(True, linestyle='-', color = '0.75')
if save_verb: plt.savefig("./results/dimension_analysis/mean_diameter_trackpy.png", bbox_inches='tight')
if show_verb:
    plt.show()
else:
    plt.close()

In [ ]:
# windowed ?
nFrames = len(mean_dim)
print(nFrames)
# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")

# mean and std droplet diameter per frame
mean_d_wind = np.zeros(nSteps)
std_d_wind = np.zeros(nSteps)
for i, start in enumerate(startFrames):
    mean_d_wind[i] = np.mean(2*mean_dim["size"][start:start+window])
    std_d_wind[i] = np.std(2*mean_dim["size"][start:start+window])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 5))
ax.plot(startFrames/10, mean_d_wind)
ax.vlines(merge_frame/10, mean_d_wind.min(), mean_d_wind.max(), color="r")
ax.fill_between(startFrames/10, mean_d_wind - std_d_wind, mean_d_wind + std_d_wind, alpha=0.5)
ax.set(xlabel = "Window Time [s]", ylabel = "d [px]", title = "Droplet diameter over window time")
ax.grid(True, linestyle='-', color = '0.75')
plt.show()

## confront trackpy and hough circle results

# LINK HOUGH RESULT TO TRACKPY RESULT

In [5]:
data = hough_preprocessing(pims.open('./data/movie.mp4'), 40, 55, 895, 910)
nFrames = len(data)
merge_frame = 32269

deprecated pixel format used, make sure you did set range correctly


In [ ]:
trackpy_df = pd.read_parquet("./results/tracking_data/pre_merge_tracking_sorted_and_colored.parquet")
colors = trackpy_df.loc[trackpy_df.frame == 0].color.values

hough_df = pd.read_parquet("./results/tracking_data/hough/hough_pre_merge.parquet").replace(0, np.nan)
hough_df.loc[:49, ["frame"]] = 0
hough_df = hough_df.loc[hough_df.frame.between(0, max(trackpy_df.frame)), :]
hough_df["particle"] = np.zeros(len(hough_df), dtype=int)
pre_merge_err_frames = np.where(hough_df.groupby("frame").mean().x.isna())[0]

In [ ]:
# associate at each frame droplet ID from trackpy to the one from hough circles by minimizing distance matrix
for frame in tqdm(range(max(trackpy_df.frame))):
    # neglect frames with error in hough circle detection
    if frame in pre_merge_err_frames:
        continue
    hoguh_frame = hough_df.loc[hough_df.frame == frame]
    trackpy_frame = trackpy_df.loc[trackpy_df.frame == frame]
    dist = distance_matrix(hoguh_frame[["x", "y"]].values, trackpy_frame[["x", "y"]].values)
    row_ind, col_ind = linear_sum_assignment(dist)
    hough_df.loc[hough_df.frame == frame, ["particle"]] = trackpy_frame.loc[:, ["particle"]].values[col_ind,:]

c = []
for p in hough_df.particle:
    c.append(colors[p])
hough_df["color"] = c
hough_df.sort_values(by=["frame", "particle"], inplace=True)
if 0: hough_df.to_parquet("./results/parquet/tracking_hough_trackpy_linking.parquet")

In [ ]:
frame = 100
hough_temp = hough_df.loc[(hough_df.frame == frame)] #  & (hough_df.particle == 10)
trackpy_temp = trackpy_df.loc[(trackpy_df.frame == frame) ] #  & (trackpy_df.particle == 10)

fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 4))
ax.imshow(data[frame])
#ax.scatter(trackpy_temp.x.values, trackpy_temp.y.values, color = trackpy_temp.color.values)
ax.set(title = "Trackpy result")
ax1.imshow(data[frame])
#ax1.scatter(hough_temp.x.values, hough_temp.y.values, color = hough_temp.color.values)
ax1.set(title = "Hough result")

for i in range(50):
    ax.add_artist(plt.Circle((trackpy_temp.x.values[i],\
                               trackpy_temp.y.values[i]), \
                               2*trackpy_temp["size"].values[i],\
                               color = trackpy_temp.color.values[i], fill=False, linewidth=1))
    
    ax1.add_artist(plt.Circle((hough_temp.x.values[i], \
                              hough_temp.y.values[i]),\
                              hough_temp.d.values[i],\
                              color = hough_temp.color.values[i], fill=False, linewidth=1))

plt.show()

In [ ]:
fig = plt.figure(figsize = (5, 5))
#fig.subplots_adjust(left=0, right=1, bottom=0, top=1, wspace=None, hspace=None)
anim_running = True

def onClick(event):
    global anim_running
    if anim_running:
        ani.event_source.stop()
        anim_running = False
    else:
        ani.event_source.start()
        anim_running = True

def update_graph(frame):
    df = hough_df.loc[(hough_df.frame == frame) , ["x","y","color","d"]]
    for i in range(50):
        graph[i].center = (df.x.values[i], df.y.values[i])
        graph[i].radius = df.d.values[i]
    graph2.set_data(data[frame])
    title.set_text('Hough features location & Trackpy linking - frame = {}'.format(frame))
    return graph

ax = fig.add_subplot(111)
title = ax.set_title('Hough features location & Trackpy linking - frame = 0')
ax.set(xlabel = 'X [px]', ylabel = 'Y [px]')
df = hough_df.loc[(hough_df.frame == 0), ["x","y","color","d"]]

graph = []
for i in range(50):
    graph.append(ax.add_artist(plt.Circle((df.x.values[i], df.y.values[i]), df.d.values[i], color = df.color.values[i],\
                                           fill = False, linewidth=1)))
graph2 = ax.imshow(data[0])

fig.canvas.mpl_connect('button_press_event', onClick)
ani = matplotlib.animation.FuncAnimation(fig, update_graph, range(0, int(max(hough_df.frame)), 1), interval = 5, blit=False)
if 0: 
    writer = matplotlib.animation.FFMpegWriter(fps = 30, metadata = dict(artist='Matteo Scandola'), extra_args=['-vcodec', 'libx264'])
    ani.save('./results/tracking_hough_trackpy_linking.mp4', writer=writer, dpi = 300)
plt.show()

# TRACKPY TRACKING ERROR ESTIMATION

I note that between frames 18300 and 18900 a droplet 40 seems stable and isolated.\
Then applying a rectangular mask around it I compute the "benchmark" position of the droplet using Canny Edge Detection and Hough Transform.\
Finally I confront the tracking result with the benchmark and different window sizes of smoothing.

In [ ]:
# apply mask to perform edge detection only on the benchmark particle
@pims.pipeline
def crop(image, x1, y1, x2, y2):   
    npImage = np.array(image)
    alpha = Image.new('L', (920, 960), 0)
    draw = ImageDraw.Draw(alpha)
    draw.rectangle([(x1, y1), (x2, y2)], fill = 255)
    npAlpha = np.array(alpha)
    npImage = npImage[:, :, 1] * npAlpha
    return npImage

In [ ]:
startFrame = 18300
endFrame = 18900
framesArray = np.arange(startFrame, endFrame, 1) 

rawTrajs = pd.read_parquet("./results/tracking_data/trackpy_pre_merge.parquet")
benchmark_particle_id = 40
print("Benchmark particle:", benchmark_particle_id)
rawTraj = rawTrajs.loc[(rawTrajs.frame.between(startFrame, endFrame-1)) & (rawTrajs.particle == benchmark_particle_id)]
if 1: ref = list(pims.open('./data/movie.mp4')[startFrame:endFrame])

In [ ]:
# apply mask to perform edge detection only on the benchmark particle
y1 = rawTraj.iloc[0].y - 50
x1 = rawTraj.iloc[0].x - 50
y2 = rawTraj.iloc[0].y + 110
x2 = rawTraj.iloc[0].x + 120
if 1: ref_masked = list(crop(pims.open('./data/movie.mp4'), x1, y1, x2, y2)[startFrame:endFrame])

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (8, 6))
ax.imshow(ref[0])
ax.scatter(rawTraj.iloc[0].x, rawTraj.iloc[0].y, s=150, facecolors='none', edgecolors='b')
ax.set(title = "Selection of benchmark particle", xlabel = "x [px]", ylabel = "y [px]")
ax1.imshow(ref_masked[0])
ax1.scatter(rawTraj.iloc[0].x, rawTraj.iloc[0].y, s=150, facecolors='none', edgecolors='b')
ax1.set(title = "Masked video", xlabel = "x [px]")
plt.tight_layout()
plt.savefig("./results/error_estimation/masked_frame.png", bbox_inches='tight')
plt.show()

In [ ]:
# needed to check that only particle 40 is selected by the mask
fig = plt.figure()
anim_running = True

def onClick(event):
    global anim_running
    if anim_running:
        ani.event_source.stop()
        anim_running = False
    else:
        ani.event_source.start()
        anim_running = True

def update_graph(frame):
    df = rawTraj.loc[rawTraj.frame == startFrame + frame, ["x","y", "color"]]
    graph.set_offsets(df)
    graph2.set_data(ref_masked[frame])
    title.set_text('frame = {}'.format(startFrame + frame))
    return graph

ax = fig.add_subplot(111)
title = ax.set_title(f'frame = {startFrame}')
df = rawTraj.loc[rawTraj.frame == startFrame, ["x","y", "color"]]

graph = ax.scatter(df.x, df.y, facecolors = 'none', edgecolors= df.color, s = 150)

graph2 = ax.imshow(ref_masked[0])

fig.canvas.mpl_connect('button_press_event', onClick)
ani = matplotlib.animation.FuncAnimation(fig, update_graph, endFrame-startFrame, interval = 2, blit=False)
plt.show()

In [ ]:
circles = []
problems = 0

for i in tqdm(range(0, len(ref_masked))):
    img = ref_masked[i]
    """
                    -- this works fine !!!!! --
    # Apply Canny Edge Detection to find edges in the image
    edges = cv2.Canny(img, 30, 30)
    # Apply the Hough Transform to find circles in the image 
    temp = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, minDist=20, param1=50, param2=30, minRadius=0, maxRadius=0)
    if temp is not None:
        circles.append(temp[0][0])
   """
    
    # this gives better results
    temp = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, 1.5, minDist=1, param1=300, param2=0.6, minRadius=10, maxRadius=21)
    if temp is not None:
        circles.append(temp[0][0])
        
    else:
        break
        problems += 1
print("Number of problems:", problems)

In [ ]:
x = np.zeros(len(circles))
y = np.zeros(len(circles))
d = np.zeros(len(circles))

for i in range(len(circles)):
    x[i] = circles[i][0]
    y[i] = circles[i][1]
    d[i] = circles[i][2]
    
benchmark_traj = pd.DataFrame({"x": x, "y": y, "d": d})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 6))
ax.plot(np.arange(startFrame, endFrame, 1), benchmark_traj.d, label = "benchmark")
ax.plot(np.arange(startFrame, endFrame, 1), 2*rawTraj["size"], label = "traj")
ax.legend()
plt.show()

In [ ]:
c1 = plt.Circle(( x[100] , y[100] ), d[100], fill = False, color = "red")
c2 = plt.Circle((rawTraj.x.values[100], rawTraj.y.values[100]), 2*rawTraj["size"].values[100], fill = False, color="blue")

fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 8))
ax.imshow(ref_masked[100])
ax.add_artist(c1)
ax.scatter(x[100], y[100], color = "red")
ax.set(xlim = (x1, x2), ylim = (y1, y2))
ax1.imshow(ref_masked[100])
ax1.add_artist(c2)
ax1.scatter(rawTraj.x.values[100], rawTraj.y.values[100], color = "blue")
ax1.set(xlim = (x1, x2), ylim = (y1, y2))
plt.show()

## POSITION ERROR

In [ ]:
tracking_pos = np.sqrt(rawTraj.x**2 + rawTraj.y**2)

benchmark_pos = np.sqrt(x**2 + y**2)
mse_raw = ((benchmark_pos - tracking_pos)**2).mean()
print("MSE with raw trajectory:", mse_raw)

In [ ]:
from scipy.signal import savgol_filter
def get_smooth_trajs(trajs, nDrops, windLen, orderofPoly):
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    ret = trajs.copy()
    for i in range(nDrops):
        ret.loc[ret.particle == i, "x"] = savgol_filter(trajs.loc[trajs.particle == i].x.values, windLen, orderofPoly)
        ret.loc[ret.particle == i, "y"] = savgol_filter(trajs.loc[trajs.particle == i].y.values, windLen, orderofPoly)    
    return ret

In [ ]:
windLen = 30
smoothTrajs = get_smooth_trajs(rawTrajs, 50, windLen, 2)
smoothTraj = smoothTrajs.loc[(smoothTrajs.frame.between(startFrame, endFrame-1)) & (smoothTrajs.particle == benchmark_particle_id)]
tracking_smooth_pos = np.sqrt(smoothTraj.x**2 + smoothTraj.y**2)
mse = ((benchmark_pos - tracking_smooth_pos)**2).mean()
print(f"MSE smoothing window {windLen}:", mse)

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 5))
ax.plot(framesArray, benchmark_pos, label = "benchmark")
ax.plot(framesArray, tracking_pos, label = "tracking")
ax.set_title("Confront with raw traj")
ax1.plot(framesArray, benchmark_pos, label = "benchmark")
ax1.plot(framesArray, tracking_smooth_pos, label = "tracking")
ax1.set_title(f"Confront with smooth traj (window = {windLen})")
ax.legend()
ax1.legend()
plt.tight_layout()
plt.savefig("./results/error_estimation/benchmark_confront_wind30.png", bbox_inches='tight')
plt.show()

### MSE ANALYSIS OF THE SMOOTHING WINDOW

In [ ]:
windLenList = np.arange(3, 100, 1)
mse = np.zeros(len(windLenList))

for k in tqdm(range(len(windLenList))):
    smoothTrajs = get_smooth_trajs(rawTrajs, 50, windLenList[k], 2)
    smoothTraj = smoothTrajs.loc[(smoothTrajs.frame.between(startFrame, endFrame-1)) & (smoothTrajs.particle == benchmark_particle_id)]
    tracking_smooth_pos = np.sqrt(smoothTraj.x**2 + smoothTraj.y**2)
    mse[k] = ((benchmark_pos - tracking_smooth_pos)**2).mean()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 6))
ax.hlines(mse_raw, windLenList[0], windLenList[-1], 'r', label = "raw")
ax.plot(windLenList, mse, label = "smooth")
ax.set_title("MSE vs smoothing window")
ax.set_xlabel("Smoothing window")
ax.set_ylabel("MSE")
ax.legend(loc='center right')
plt.savefig("./results/error_estimation/mse_vs_window.png", bbox_inches='tight')
plt.show()

# MERGING ANALYSIS

In [ ]:
preMerge = data[32268]
merge = data[32269]
postMerge = data[32270]

# feature location with minMass, have some problems but the spurious effect are solved
f = tp.locate(preMerge, dropSize, minmass = minMass, separation = sep, topn = nDrops, engine = 'numba')

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(f['mass'], bins = 20)
ax1.set(xlabel='mass', ylabel='count')
ax2.imshow(preMerge)
ax2.plot(f.x, f.y, 'bo')
plt.suptitle(f"Number of features found: {len(f)}")
plt.show()

f = tp.locate(merge, dropSize, minmass = minMass, separation = sep, topn = nDrops-1, engine = 'numba')
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(f['mass'], bins = 20)
ax1.set(xlabel='mass', ylabel='count')
ax2.imshow(merge)
ax2.plot(f.x, f.y, 'bo')
plt.suptitle(f"Number of features found: {len(f)}")

f = tp.locate(postMerge, dropSize, minmass = minMass, separation = sep, topn = nDrops-1, engine = 'numba')
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(f['mass'], bins = 20)
ax1.set(xlabel='mass', ylabel='count')
ax2.imshow(postMerge)
ax2.plot(f.x, f.y, 'bo')
plt.suptitle(f"Number of features found: {len(f)}")

plt.show()
tp.subpx_bias(f)
plt.show()